In [1]:
import numpy as np
from AbstractMesh import AbstractMesh
import utils
from metrics import quad_area, quad_aspect_ratio
from scipy.sparse import *

In [2]:
class Quadmesh(AbstractMesh):
    def __init__(self, vertices= None, faces = None, face_normals = None, labels = None):
        
        self.face_normals    = None #npArray (Nx3)
        self.face_labels     = None #npArray (Nx1)
        self.faces           = None #npArray (Nx4)
        self.__face2face     = None #npArray (Nx3?)
        
        super(Quadmesh, self).__init__()
        
        if vertices and faces:
            self.vertices = np.array(vertices)
            self.faces = np.array(faces)
            
            if(faces_normals):
                self.face_normals = np.array(face_normals)
                
            if(labels):
                self.labels = np.array(labels)
                            
            self.__load_operations()
            
    
    def show(self):
        pass
    
    
    @property
    def num_faces(self):
        
        return self.faces.shape[0]
       
#_______________________________Operations on Mesh Elements__________________________________________________________________________________
        
    def add_face(self, face_id0, face_id1, face_id2, face_id3):
        
        self.add_faces([face_id0, face_id1, face_id2, face_id3])
        
        
    def add_faces(self, new_faces):
        
        new_faces = np.array(new_faces)
        new_faces.shape = (-1,4)
        
        if new_faces[(new_faces[:,0] > self.num_vertices) | 
                     (new_faces[:,1] > self.num_vertices) | 
                     (new_faces[:,2] > self.num_vertices) | 
                     (new_faces[:,3] > self.num_vertices)].shape[0] > self.num_vertices:
            raise Exception('The Id of a vertex must be lesser than the number of vertices')

        self.faces = np.concatenate([self.faces, new_faces])
        self.__load_operations()
        
        
    def remove_face(self, face_id):
        
        self.remove_faces([face_id])
        
        
    def remove_faces(self, face_ids):
        
        face_ids = np.array(face_ids)
        mask = np.ones(self.num_faces)
        mask[face_ids] = 0
        mask = mask.astype(np.bool)
        
        self.faces = self.faces[mask]
        self.__load_operations()
        
        
    def remove_vertex(self, vtx_id):
        
        self.remove_vertices([vtx_id])
                
        
    def remove_vertices(self, vtx_ids):
        
        vtx_ids = np.array(vtx_ids)
        
        for v_id in vtx_ids:
            
            self.vertices = np.delete(self.vertices, v_id, 0)
            self.faces = self.faces[(self.faces[:,0] != v_id) & 
                                    (self.faces[:,1] != v_id) & 
                                    (self.faces[:,2] != v_id) &
                                    (self.faces[:,3] != v_id)]
            
            self.faces[(self.faces[:,0] > v_id)] -= np.array([1, 0, 0, 0])
            self.faces[(self.faces[:,1] > v_id)] -= np.array([0, 1, 0, 0])
            self.faces[(self.faces[:,2] > v_id)] -= np.array([0, 0, 1, 0])
            self.faces[(self.faces[:,3] > v_id)] -= np.array([0, 0, 0, 1])
            
            vtx_ids[vtx_ids > v_id] -= 1;
        
        self.__load_operations()
        
#____________________________________________________________________________________________________________________________________________________
        
    def __compute_adjacencies(self):
        
        map_ = dict()
        adjs =  [[] for i in range(self.num_faces)]
        vtx2vtx = [[] for i in range(self.num_vertices)]
        vtx2face = [[] for i in range(self.num_vertices)]


        edges = np.c_[self.faces[:,0], self.faces[:,1], 
                      self.faces[:,1], self.faces[:,2], 
                      self.faces[:,2], self.faces[:,3], 
                      self.faces[:,3], self.faces[:,0]]
        edges.shape = (-1, 2)
        faces_idx = np.repeat(np.array(range(self.num_faces)), 4)

        for e, f in zip(edges, faces_idx):
            
            vtx2vtx[e[0]].append(e[1])
            vtx2face[e[0]].append(f)
            vtx2face[e[1]].append(f)
            e = (e[0], e[1])

            try:
                tmp = map_[e]
            except KeyError:
                tmp = None

            if tmp is None:
                map_[(e[1], e[0])] = f
            else:
                adjs[f].append(map_[e])
                adjs[map_[e]].append(f)

        self.__face2face = np.array([np.array(a) for a in adjs])
        self._AbstractMesh__vtx2vtx = np.array([np.array(a) for a in vtx2vtx])
        self._AbstractMesh__vtx2face = np.array([np.unique(np.array(a)) for a in vtx2face])

    
    def __load_operations(self):
        
        self.__compute_adjacencies()
        self._AbstractMesh__update_bounding_box()
        self.__compute_face_normals()
        self.__compute_vertex_normals()
        self.__compute_metrics()
        
    
    def __compute_face_normals(self):
        
        e1_v = self.vertices[self.faces][:,1] - self.vertices[self.faces][:,0]
        e2_v = self.vertices[self.faces][:,2] - self.vertices[self.faces][:,0]
        
        self.face_normals = np.cross(e1_v, e2_v)
        norm = np.linalg.norm(self.face_normals, axis=1)
        norm.shape = (-1,1)
        self.face_normals /= norm
        
    def __compute_vertex_normals(self):
        
        self.vtx_normals = np.array([np.mean(self.face_normals[v2f], axis=0) for v2f in self.vtx2face])
        norm = np.linalg.norm(self.vtx_normals, axis=1)
        norm.shape = (-1,1)
        self.vtx_normals = self.vtx_normals / norm
        
        
    def load_from_file(self, filename):
        
        ext = filename.split('.')[-1]
        
        if ext == 'obj':
            self.vertices, self.faces, self.face_normals = utils.read_obj(filename)

        self.__load_operations()
        
        
    def save_file(self, filename):
        
        ext = filename.split('.')[-1]
        
        if ext == 'obj':
            utils.save_obj(self, filename)
    
    
    @property
    def simplex_centroids(self):
        
        if self._AbstractMesh__simplex_centroids is None:
            self._AbstractMesh__simplex_centroids = self.vertices[self.faces].mean(axis = 1)
        
        return self._AbstractMesh__simplex_centroids
    
    
    @property
    def face2face(self):
        
        return self.__face2face
    
    def __compute_metrics(self):
        
        self.simplex_metrics['area'] = quad_area(self.vertices, self.faces)
        self.simplex_metrics['aspect_ratio'] = quad_aspect_ratio(self.vertices, self.faces)
        
    def boundary(self, flip_x = False, flip_y = False, flip_z = False):
        
        min_x = self.cut['min_x']
        max_x = self.cut['max_x']
        min_y = self.cut['min_y']
        max_y = self.cut['max_y']
        min_z = self.cut['min_z']
        max_z = self.cut['max_z']
            
        x_range = np.logical_xor(flip_x,((self.simplex_centroids[:,0] >= min_x) & (self.simplex_centroids[:,0] <= max_x)))
        y_range = np.logical_xor(flip_y,((self.simplex_centroids[:,1] >= min_y) & (self.simplex_centroids[:,1] <= max_y)))
        z_range = np.logical_xor(flip_z,((self.simplex_centroids[:,2] >= min_z) & (self.simplex_centroids[:,2] <= max_z)))
        
        return self.faces[ x_range & y_range & z_range]
        
    
    @property
    def export_triangles(self):
        
        tris = np.c_[self.faces[:,:3], self.faces[:,2:], self.faces[:,0]]
        tris.shape = (-1, 3)
        return tris

In [3]:
qm = Quadmesh()
qm.load_from_file('sample.obj')

In [6]:
qm.face_normals[2162:2168]

array([[-3.22412365e-02, -8.93006898e-01, -4.48886604e-01],
       [-6.32914182e-03, -8.66897235e-01, -4.98446713e-01],
       [ 4.74447442e-03, -8.48620531e-01, -5.28980797e-01],
       [ 4.32015588e-01, -6.86352853e-01, -5.85048967e-01],
       [ 9.82676488e-01,  1.62102731e-01, -8.98310856e-02],
       [ 9.96583449e-01,  8.03091834e-02, -1.92837762e-02],
       [-8.31704566e-01, -3.49294205e-01, -4.31579741e-01],
       [ 9.99314033e-01,  2.99407887e-04, -3.70320709e-02],
       [-8.34545734e-01, -3.08986273e-01, -4.56136932e-01],
       [ 9.99762903e-01,  1.35834415e-02, -1.70184715e-02],
       [-8.44743563e-01, -2.85204135e-01, -4.52843146e-01],
       [ 9.99478208e-01, -1.23153738e-02, -2.98603945e-02],
       [-8.62618624e-01, -2.02935485e-01, -4.63364110e-01],
       [ 9.99514147e-01,  2.10074000e-02, -2.30251993e-02],
       [-8.80552452e-01, -1.21576074e-01, -4.58090207e-01],
       [ 9.99728923e-01, -7.21236546e-03, -2.21373336e-02],
       [-8.80366901e-01, -4.37211439e-02

In [38]:
qm.vtx_normals

array([[-0.57735027, -0.57735027, -0.57735027],
       [-0.57735027,  0.57735027, -0.57735027],
       [ 0.57735027,  0.57735027, -0.57735027],
       [ 0.57735027, -0.57735027, -0.57735027],
       [-0.57735027, -0.57735027,  0.57735027],
       [-0.57735027,  0.57735027,  0.57735027],
       [ 0.57735027,  0.57735027,  0.57735027],
       [ 0.57735027, -0.57735027,  0.57735027]])

array([[   2,    1,    0],
       [   0,    3,    2],
       [   5,    4,    0],
       ...,
       [2408, 2409, 2411],
       [2413, 2412, 2410],
       [2410, 2411, 2413]])